In [20]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId = 33)

<IB connected to 127.0.0.1:7497 clientId=33>

In [21]:
ib.portfolio()

[]

Peer closed connection


In [2]:
# Place a market order of 100 shares of GOOG
contract = Stock('GOOG', 'SMART', 'USD')
ib.qualifyContracts(contract)

order = MarketOrder('BUY', 100)

In [3]:
trade = ib.placeOrder(contract, order)
trade

Trade(contract=Stock(conId=208813720, symbol='GOOG', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='GOOG', tradingClass='NMS'), order=MarketOrder(orderId=26, clientId=13, action='BUY', totalQuantity=100), orderStatus=OrderStatus(orderId=26, status='PendingSubmit', filled=0, remaining=0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2021, 2, 17, 21, 15, 56, 334333, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

In [4]:
ib.sleep(1)
trade.log

[TradeLogEntry(time=datetime.datetime(2021, 2, 17, 15, 35, 15, 65198, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''),
 TradeLogEntry(time=datetime.datetime(2021, 2, 17, 15, 35, 15, 679433, tzinfo=datetime.timezone.utc), status='Submitted', message=''),
 TradeLogEntry(time=datetime.datetime(2021, 2, 17, 15, 35, 15, 740566, tzinfo=datetime.timezone.utc), status='Submitted', message='Fill 100.0@2115.0'),
 TradeLogEntry(time=datetime.datetime(2021, 2, 17, 15, 35, 15, 741568, tzinfo=datetime.timezone.utc), status='Filled', message='')]

In [5]:
assert trade in ib.trades()

In [6]:
assert order in ib.orders()

In [3]:
import datetime
datetime.datetime.now()

datetime.datetime(2021, 2, 18, 5, 16, 44, 185733)

In [4]:
starttime = datetime.datetime(2021, 2, 15, 10, 0, 0)
starttime

datetime.datetime(2021, 2, 15, 10, 0)

In [ ]:
# A) Place 10 market orders in 10 minutes increment starting at 10:00am in a given day, with each order to buy 10 shares of GOOG.

# Write a dead loop, a slippage will occur.
i = 0
while True:
    if datetime.datetime.now() == (starttime + i * datetime.timedelta(minutes = 10)):
        order = MarketOrder('BUY', 10)
        trade = ib.placeOrder(contract, order)
        trade
        i = i + 1
    if (i == 10):
        break
# B) Place the same amount of sell orders to flatten out your positions.
order = MarketOrder('SELL', 100)
trade = ib.placeOrder(contract, order)
trade  

    
# Another way is using ib.sleep().
td = datetime.now()
i = 0
if [td.hour, td.minute] == [10, 0]:
    while(i < 10):
        order = MarketOrder('BUY', 10)
        ib.placeOrder(contract, order)
        i = i + 1
        ib.sleep(600) # 60 seconds * 10 = 600 seconds
        print("placing", i, "-th order")

In [9]:
lmtPrice = ib.reqMktData(contract, 221)
print(lmtPrice.markPrice)

2129.75


In [11]:
ib.reqPositions()

[Position(account='DU3225639', contract=Stock(conId=208813720, symbol='GOOG', exchange='NASDAQ', currency='USD', localSymbol='GOOG', tradingClass='NMS'), position=0.0, avgCost=0.0)]

In [ ]:
# You believe the price of GOOG is going down from here. So, you would like to place limit orders to buy as price drops. 
# Specifically, you would like to buy 20 shares of GOOG for every price drop of 0.50% from the previous limit price, 
# but you want to cap the maximal shares to hold at 100 shares.

lmtPrice = ib.reqMktData(contract, 221).markPrice
num = 0
while num <= 100:
    limitOrder = LimitOrder('BUY', 20, lmtPrice)
    limitTrade = ib.placeOrder(contract, limitOrder)
    lmtPrice = (1-0.005) * lmtPrice
    num = num + 20
